In [ ]:
pip install huggingface_hub[hf_xet]

In [1]:
from transformers import pipeline
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
input = '../data/data_intermediaire_test.parquet'

In [3]:
df = pd.read_parquet(input)

In [5]:
classifier = pipeline(
    "sentiment-analysis", 
    model="distilbert/distilbert-base-uncased-finetuned-sst-2-english",
    device=0,
    truncation=True
    )

Device set to use cpu


In [6]:
results = []

batch_size = 50

for i in tqdm(range(0, len(df), batch_size), desc="Analyse de sentiments"):
    batch_texts = df['text'].iloc[i:i+batch_size].tolist()
    batch_results = classifier(batch_texts, truncation=True)  
    results.extend(batch_results)

Analyse de sentiments: 100%|██████████| 500/500 [1:12:59<00:00,  8.76s/it]


In [7]:
df['predict'] = [1 if res['label'] == 'POSITIVE' else 0 for res in results]

In [19]:
accuracy = accuracy_score(df['label'], df['predict'])
print(f"✅ Accuracy: {accuracy:.2%}")

cm = confusion_matrix(df['label'], df['predict'])
print(cm)

✅ Accuracy: 89.07%
[[11494  1006]
 [ 1726 10774]]
